In [22]:
import json
import urllib3
import config
import pandas as pd
import string
import datetime

http = urllib3.PoolManager()

# http://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}
location = [
    'New York City, NY, USA',
    'Los Angeles, CA, USA',
    'Chicago, IL, USA',
    'Houston, TX, USA',
    'Phoenix, AZ, USA',
    'Philadelphia, PA, USA',
    'San Antonio, TX, USA',
    'San Diego, CA, USA',
    'Dallas, TX, USA',
    'San Jose, CA, USA',
    'Austin, TX, USA',
    'Jacksonville, FL, USA',
    'Fort Worth, TX, USA',
    'Columbus, OH, USA',
    'San Francisco, CA, USA',
    'Charlotte, NC, USA',
    'Indianapolis, IN, USA',
    'Seattle, WA, USA',
    'Denver, CO, USA',
    'Washington, DC, USA'    
]

# append to empty df doesn't happen in place; need to store it first
# https://stackoverflow.com/questions/16597265/appending-to-an-empty-dataframe-in-pandas
loc_responses = []
all_weather_data = []
weather_data = []

for l in location:
    # Location call
    call_string = 'https://api.openweathermap.org/geo/1.0/direct?q=' + l + '&limit=1&appid=' + config.api_key
    loc_response = json.loads(http.request('GET', call_string).data)
    loc_responses.append(loc_response)
    print(l, loc_response[0]['lat'], loc_response[0]['lon'])

    # Weather call
    call_string = 'https://api.openweathermap.org/data/2.5/weather?lat=' + str(loc_response[0]['lat']) + '&lon=' + str(loc_response[0]['lon']) + '&appid=' + config.api_key
    weather_response = json.loads(http.request('GET', call_string).data)
    all_weather_data.append(weather_response)
    
    # optional additional weather information:
    add_cols = [
        ['humidity' , 'main' , 'humidity', 1],
        ['cloudiness' , 'clouds' , 'all', 1],
        ['wind_speed' , 'wind' , 'speed', 2.237],
        ['rain_1h' , 'rain' , '1h', 1/25.4],
        ['rain_3h' , 'rain' , '3h', 1/25.4],
        ['snow_1h' , 'snow' , '1h', 1/25.4],
        ['snow_3h' , 'snow' , '3h', 1/25.4]
    ]

    for i in add_cols:
        try : 
            i.append(weather_response[i[1]][i[2]] * i[3])
        except : 
            i.append(0)
    
    # useful for processing json: https://stackoverflow.com/questions/67745643/select-specific-keys-inside-a-json-using-python
    weather_trim = { 
        'city' : weather_response['name'],
        'description' : weather_response['weather'][0]['description'],
        'icon' : weather_response['weather'][0]['icon'],   
        'current_temp' : weather_response['main']['temp'] - 273.15,
        'feels_like' : weather_response['main']['feels_like'] - 273.15,
        'temp_min' : weather_response['main']['temp_min'] - 273.15,
        'temp_max' : weather_response['main']['temp_max'] - 273.15,
        add_cols[0][0] : add_cols[0][4],
        add_cols[1][0] : add_cols[1][4],
        add_cols[2][0] : add_cols[2][4],
        add_cols[3][0] : add_cols[3][4],
        add_cols[4][0] : add_cols[4][4],
        add_cols[5][0] : add_cols[5][4],
        add_cols[6][0] : add_cols[6][4],
        #'visibility': weather_response['visibility'],
        'sunrise' : datetime.datetime.fromtimestamp(weather_response['sys']['sunrise']),
        'sunset' : datetime.datetime.fromtimestamp(weather_response['sys']['sunset']),
        'current_time' :  datetime.datetime.fromtimestamp(weather_response['dt']),
        'timezone' : weather_response['timezone'],
        'lat' : weather_response['coord']['lat'],
        'lon' : weather_response['coord']['lon']        
    }

    weather_data.append(weather_trim)

New York City, NY, USA 40.7127281 -74.0060152
Los Angeles, CA, USA 34.0536909 -118.242766
Chicago, IL, USA 41.8755616 -87.6244212
Houston, TX, USA 29.7589382 -95.3676974
Phoenix, AZ, USA 33.4484367 -112.074141
Philadelphia, PA, USA 39.9527237 -75.1635262
San Antonio, TX, USA 29.4246002 -98.4951405
San Diego, CA, USA 32.7174202 -117.1627728
Dallas, TX, USA 32.7762719 -96.7968559
San Jose, CA, USA 37.3361663 -121.890591
Austin, TX, USA 30.2711286 -97.7436995
Jacksonville, FL, USA 30.3321838 -81.655651
Fort Worth, TX, USA 32.753177 -97.3327459
Columbus, OH, USA 39.9622601 -83.0007065
San Francisco, CA, USA 37.7790262 -122.419906
Charlotte, NC, USA 35.2272086 -80.8430827
Indianapolis, IN, USA 39.7683331 -86.1583502
Seattle, WA, USA 47.6038321 -122.330062
Denver, CO, USA 39.7392364 -104.984862
Washington, DC, USA 38.8950368 -77.0365427


In [23]:
weather_df = pd.json_normalize(weather_data)
weather_df.head()

,city,description,icon,current_temp,feels_like,temp_min,temp_max,humidity,cloudiness,wind_speed,rain_1h,rain_3h,snow_1h,snow_3h,sunrise,sunset,current_time,timezone,lat,lon
0,New York,clear sky,01d,31.71,34.07,29.79,33.50,51,0,10.35731,0,0,0,0,2024-07-14 02:37:06,2024-07-14 17:26:21,2024-07-14 10:12:50,-14400,40.7127,-74.0060
1,Los Angeles,haze,50d,22.72,23.06,20.01,26.75,77,0,6.91233,0,0,0,0,2024-07-14 05:52:15,2024-07-14 20:05:08,2024-07-14 10:09:34,-25200,34.0537,-118.2428
2,Chicago,broken clouds,04d,26.82,28.56,25.03,27.75,70,75,13.80229,0,0,0,0,2024-07-14 03:28:00,2024-07-14 18:24:24,2024-07-14 10:12:56,-18000,41.8756,-87.6244
3,Houston,broken clouds,04d,32.30,39.30,31.03,33.95,70,75,5.74909,0,0,0,0,2024-07-14 04:30:42,2024-07-14 18:23:40,2024-07-14 10:15:31,-18000,29.7589,-95.3677
4,Phoenix,scattered clouds,03d,35.31,38.45,33.92,36.53,42,40,8.05320,0,0,0,0,2024-07-14 05:29:02,2024-07-14 19:38:59,2024-07-14 10:08:04,-25200,33.4484,-112.0740


In [24]:
# Weather forecast
# api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}
all_forecasts = []
forecast_data = []

for l in loc_responses:
    call_string = 'https://api.openweathermap.org/data/2.5/forecast?lat=' + str(l[0]['lat']) + '&lon=' + str(l[0]['lon']) + '&appid=' + config.api_key
    forecast_response = json.loads(http.request('GET', call_string).data)
    all_forecasts.append(forecast_response)
    for l in forecast_response['list']:
        forecast_trim = { 
            'city' : forecast_response['city']['name'],
            'description' : l['weather'][0]['description'],
            'icon' : l['weather'][0]['icon'],   
            'current_temp' : l['main']['temp'] - 273.15,
            'feels_like' : l['main']['feels_like'] - 273.15,
            'temp_min' : l['main']['temp_min'] - 273.15,
            'temp_max' : l['main']['temp_max'] - 273.15,
            #'visibility': l['visibility'],
            'forecast_time' :  datetime.datetime.fromtimestamp(l['dt']),
            'timezone' : forecast_response['city']['timezone']
        }
        forecast_data.append(forecast_trim)

In [25]:
forecast_df = pd.json_normalize(forecast_data)
forecast_df.head()

,city,description,icon,current_temp,feels_like,temp_min,temp_max,forecast_time,timezone
0,New York,clear sky,01d,31.72,34.09,31.72,32.50,2024-07-14 11:00:00,-14400
1,New York,scattered clouds,03d,30.78,33.05,28.89,30.78,2024-07-14 14:00:00,-14400
2,New York,light rain,10d,28.93,31.22,27.54,28.93,2024-07-14 17:00:00,-14400
3,New York,light rain,10n,26.05,26.05,26.05,26.05,2024-07-14 20:00:00,-14400
4,New York,light rain,10n,25.12,25.75,25.12,25.12,2024-07-14 23:00:00,-14400


In [26]:
final_df = forecast_df.merge(weather_df, on = 'city', how = 'inner',  suffixes=('_forecast', '_current'))
final_df.to_excel('current_forecast_weather.xlsx')